# Analysis of Motion Sickness and Embodiment Data

Author: Zora Nolte

Last updated: 12.02.2022

In [ ]:
import os
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.factorplots import interaction_plot
import numpy as np
from scipy import stats

In [ ]:
# load the data
search_path = os.getcwd() + '\\data\\' # location of the data files
df = pd.read_csv(search_path + "data_questionnaires_preprocessed.csv", converters={'BE04_01':str, 'BE04_02':str, 'BE04_03':str})
df = df.rename(columns={"blob" : 'condition'})

In [ ]:
sig_level = 0.05 # p value to be used as threshold

## Notes on Mixed ANOVA

**Independent variables:** two categorical independent variables (factors)
- one of which is between-subject (each subjects assigned only once to treatment): blob vs. avatar
- the other is within-subject (each subject assigned multiple treatments): hybrid vs. first-person

**Assumptions:**
- The responses from subjects (dependent variable) should be continuous
- Residuals (experimental error) are approximately normally distributed for each combination of between-subject and within-subject variable (Shapiro-Wilks Test or histogram)
- Homogeneity of variances or homoscedasticity: There should be equal variance for every level of within-subject factor (Levene’s test)
- Assumption of sphericity: the variances of differences in responses between any two levels of the independent variable (within-subjects factor) should be equal (Mauchly’s test of sphericity). This assumptionn is also known as homogeneity-of-variance-of-differences assumption.
- Homogeneity of the variance-covariance matrices: the pattern of intercorrelation for each level of within-subject variable across between-subject variable should be equal. This is a multivariate version of the Homogeneity of variances. It can be tested using Box’s M test. Box’s M-test has little power and uses a lower alpha level such as 0.001 to assess the p value for significance.
- There should be no significant outlier (this can be checked by boxplot)
 
**Results:**
- Two-way mixed ANOVA estimates the three effects - two main effects and one interaction effect - for statistical significance
- Generally, it is not appropriate to interpret main effects when interaction is significant.

(Source: https://www.reneshbedre.com/blog/mixed-anova.html)


## Naming conventions

- condition: blob vs. avatar (1: blob, 0: avatar)
- mode: first-person vs. hybrid

## Important Note

**p-values are not corrected for multiple analyses in this notebook!**

## Functions

In [ ]:
def summarize(df, var):
    '''
    Groups the df by the independent factors, and prints the variable's count, mean, and std.
    
    df (pd.DataFrame)
    var (str): the column to summarize
    '''
    return df.groupby(["condition", "mode"]).describe()[var][["count", "mean", "std"]]

In [ ]:
def boxplot(df, var, title, ax, legend=True):
    '''
    Creates a boxplot of the variable, separated by condition and mode.
    
    df (pd.DataFrame)
    var (str): the column to plot
    title (str): title of the boxplot
    ax (plt.axis): ax to be plotted on
    
    '''
    my_palette = sns.color_palette([sns.color_palette("tab10")[1], sns.color_palette("tab10")[0]])
    
    sns.boxplot(x='condition', y=var, hue='mode', data=df, ax=ax,
                showmeans=True, meanprops={"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"},
                palette=my_palette)
    ax.set_title(title, fontsize=18)
    if legend:
        ax.legend(fontsize=13, title_fontsize=13, loc='upper left', bbox_to_anchor=(1, 1))
    else:
        ax.get_legend().remove()
    
    ax.xaxis.label.set_size(16)
    ax.xaxis.labelpad = 10
    ax.yaxis.label.set_size(16)
    ax.yaxis.labelpad = 10
    ax.tick_params(axis='both', which='major', labelsize=14)
    sns.despine()


In [ ]:
def interact_plot(df, var, title, ax):
    '''
    Plots the interaction of condition and mode for the variable.
    
    df (pd.DataFrame)
    var (str):  dependent variable column name
    title (str): title of the plot
    '''
    interaction_plot(x=df['condition'], trace=df['mode'], response=df[var], ax=ax,
                     colors=[sns.color_palette("tab10")[0], sns.color_palette("tab10")[1]], 
                     markers=['D','^'])
    ax.set_title(title, fontsize=18)
    ax.legend(title="Mode", fontsize=13, title_fontsize=13, loc='upper left', bbox_to_anchor=(1, 1))
    ax.set_ylabel("Mean")
    ax.set_xlabel("Condition")
    
    ax.xaxis.label.set_size(16)
    ax.xaxis.labelpad = 10
    ax.yaxis.label.set_size(16)
    ax.yaxis.labelpad = 10
    ax.tick_params(axis='both', which='major', labelsize=14)
 
    sns.despine()
    plt.show()

In [ ]:
def check_assumptions(df, dv, dv_name):
    '''
    Checks the assumptions of a mixed ANOVA.
    
    df (pd.DataFrame)
    dv (str): dependent variable column name
    dv_name (str): name of the dependent variable (to be printed)
    '''
    
    print("Checking the assumptions for the dependent variable {}...\n".format(dv_name))
    all_assumptions = []
    
    # Assumption of the residuals being normally distributed
    df['factor_comb'] = df["condition"] + '-'+ df["mode"]
    normal = pg.normality(data=df, dv=dv, group='factor_comb')
    if normal["normal"].all():
        print("The residuals are approximately normally distributed for each level of the within-subjects factor (tested using the Shapiro-Wilks Test)")
        all_assumptions.append(True)
    else:
        print("The residuals are NOT approximately normally distributed for each level of the within-subjects factor! (tested using the Shapiro-Wilks Test)")
        all_assumptions.append(False)
    
    print()
    # Assumption of homoscedasticity
    print("There should be equal variance for every level of within-subject factor:")
    hybrid = df[df["mode"]=="Hybrid"].reset_index(drop=True)
    homo_hybrid = pg.homoscedasticity(data=hybrid, dv=dv, group='condition')
    all_assumptions.append(homo_hybrid["equal_var"][0])
    print("For the level Hybrid, Levene's test resulted in a p-value of {}, indicating that the assumption of equal variances is {}.".format(homo_hybrid["pval"][0].round(3), homo_hybrid["equal_var"][0]))
    
    fp = df[df["mode"]=="First Person"].reset_index(drop=True)
    homo_fp = pg.homoscedasticity(data=fp, dv=dv, group='condition')
    all_assumptions.append(homo_fp["equal_var"][0])
    print("For the level First Person, Levene's test resulted in a p-value of {}, indicating that the assumption of equal variances is {}.".format(homo_fp["pval"][0].round(3), homo_fp["equal_var"][0]))
    
    print()    
    # Assumption of sphericity
    sphericity = pg.sphericity(data=df, dv=dv, subject='ID', within='condition')
    all_assumptions.append(sphericity[0])
    print("Mauchly’s test of sphericity resulted in a p-value of {}, indicating that the assumption is {}.".format(sphericity[-1], sphericity[0]))
    
    print()
    # Assumption of homogeneity of the variance-covariance matrices:
    print("The homogeneity of variance-covariance matrices formed by the between-subject factor for each level of the within-subject factor should be equal:")
    box_fp = pg.box_m(data=fp, dvs=[dv], group='condition', alpha=0.001)
    all_assumptions.append(box_fp["equal_cov"][0])    
    print("For the level First Person, Box’s M test resulted in a p-value of {}, indicating that the assumption is {}.".format(box_fp["pval"][0].round(3), box_fp["equal_cov"][0]))
    
    box_hybrid = pg.box_m(data=hybrid, dvs=[dv], group='condition', alpha=0.001)
    all_assumptions.append(box_hybrid["equal_cov"][0])    
    print("For the level Hybrid, Box’s M test resulted in a p-value of {}, indicating that the assumption is {}.".format(box_hybrid["pval"][0].round(3), box_hybrid["equal_cov"][0]))

    print()
    # Assumption of no outliers
    outliers = df[df[dv] > df[dv].mean() + 3 * df[dv].std()]
    print("A data point is called an outlier if it > 3 sd away from the mean.")
    if outliers.empty:
        print("There are no outliers!")
        all_assumptions.append(True)
    else: 
        print("There are outliers! The outliers are the participants {}".format(outliers["ID"].values))
        all_assumptions.append(False)
    
    
    if all(x == True for x in all_assumptions):
        print()
        print("ALL ASSUMPTIONS ARE TRUE!")

# Motion Sickness

## Effect of condition and mode on in-game MS ratings (total score)

In [ ]:
ms = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['H_AVG_MS', 'FP_AVG_MS'])
ms = ms.rename(columns={"variable" : 'mode', "value" : "rating"})
ms.replace({'mode': {'H_AVG_MS': 'Hybrid', 'FP_AVG_MS': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

ms_anova = pg.mixed_anova(dv='rating', between='condition', within='mode', subject='ID', data=ms)
ms_anova

In [ ]:
summarize(ms, "rating")

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
boxplot(ms, "rating", "MS total score", ax)

In [ ]:
# if interaction is significant, create interaction plot
if ms_anova["p-unc"].tolist()[-1] < sig_level:
    fig, ax = plt.subplots(figsize=(4,4))
    interact_plot(ms, "rating", "MS Total Score", ax=ax)

## Effect of condition and mode on in-game MS ratings (per area)

### Area 1

In [ ]:
area_one = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['H_1_MS', 'FP_1_MS'])
area_one = area_one.rename(columns={"variable" : 'mode', "value" : "rating"})
area_one.replace({'mode': {'H_1_MS': 'Hybrid', 'FP_1_MS': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

area_one_anova = pg.mixed_anova(dv='rating', between='condition', within='mode', subject='ID', data=area_one)
area_one_anova

In [ ]:
summarize(area_one, "rating")

In [ ]:
# if interaction is significant, create interaction plot
if area_one_anova["p-unc"].tolist()[-1] < sig_level:
    fig, ax = plt.subplots(figsize=(4,4))
    interact_plot(area_one, "rating", "Interaction Condition x Mode (Area 1)", ax=ax)

### Area 2

In [ ]:
area_two = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['H_2_MS', 'FP_2_MS'])
area_two = area_two.rename(columns={"variable" : 'mode', "value" : "rating"})
area_two.replace({'mode': {'H_2_MS': 'Hybrid', 'FP_2_MS': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

area_two_anova = pg.mixed_anova(dv='rating', between='condition', within='mode', subject='ID', data=area_two)
area_two_anova

In [ ]:
summarize(area_two, "rating")

In [ ]:
# if interaction is significant, create interaction plot
if area_two_anova["p-unc"].tolist()[-1] < sig_level:
    fig, ax = plt.subplots(figsize=(4,4))
    interact_plot(area_two, "rating", "Interaction Condition x Mode (Area 2)", ax=ax)

### Area 3

In [ ]:
area_three = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['H_3_MS', 'FP_3_MS'])
area_three = area_three.rename(columns={"variable" : 'mode', "value" : "rating"})
area_three.replace({'mode': {'H_3_MS': 'Hybrid', 'FP_3_MS': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

area_three_anova = pg.mixed_anova(dv='rating', between='condition', within='mode', subject='ID', data=area_three)
area_three_anova

In [ ]:
summarize(area_three, "rating")

In [ ]:
# if interaction is significant, create interaction plot
if area_three_anova["p-unc"].tolist()[-1] < sig_level:
    fig, ax = plt.subplots(figsize=(4,4))
    interact_plot(area_three, "rating", "Interaction Condition x Mode (Area 3)", ax=ax)

### Area 4

In [ ]:
area_four = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['H_4_MS', 'FP_4_MS'])
area_four = area_four.rename(columns={"variable" : 'mode', "value" : "rating"})
area_four.replace({'mode': {'H_4_MS': 'Hybrid', 'FP_4_MS': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

area_four_anova = pg.mixed_anova(dv='rating', between='condition', within='mode', subject='ID', data=area_four)
area_four_anova

In [ ]:
summarize(area_four, "rating")

In [ ]:
# if interaction is significant, create interaction plot
if area_four_anova["p-unc"].tolist()[-1] < sig_level:
    fig, ax = plt.subplots(figsize=(4,4))
    interact_plot(area_four, "rating", "Interaction Condition x Mode (Area 4)", ax=ax)

### Plot ratings for each area

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12,8), sharey=True)
plt.subplots_adjust(hspace=0.4)
fig.suptitle('Motion Sickness Ratings', fontsize=20)

ax = ax.ravel()

boxplot(area_one, "rating", "Area 1", ax[0], legend=False)
boxplot(area_two, "rating", "Area 2", ax[1])
boxplot(area_three, "rating", "Area 3", ax[2], legend=False)
boxplot(area_four, "rating", "Area 4", ax[3], legend=False)

ax[0].set_xlabel("")
ax[1].set_xlabel("")
ax[1].set_ylabel("")
ax[3].set_ylabel("")

plt.draw()

## Simulator Sickness

Subscales:
- Oculomotor (O)
- Disorientation (D)
- Nausea (N)

## Effect of condition and mode on SSQ (total score)

In [ ]:
ssq = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['SSQ_TS_H', 'SSQ_TS_FP'])
ssq = ssq.rename(columns={"variable" : 'mode', "value" : "score"})
ssq.replace({'mode': {'SSQ_TS_H': 'Hybrid', 'SSQ_TS_FP': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

ssq_anova = pg.mixed_anova(dv='score', between='condition', within='mode', subject='ID', data=ssq)
ssq_anova

In [ ]:
summarize(ssq, "score")

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
boxplot(ssq, "score", "SSQ total score", ax)

In [ ]:
# if interaction is significant, create interaction plot
if ssq_anova["p-unc"].tolist()[-1] < sig_level:
    fig, ax = plt.subplots(figsize=(4,4))
    interact_plot(ssq, "score", "Interaction Condition x Mode (SSQ Total Score)", ax=ax)

## Assumption Check Motion Sickness

In [ ]:
check_assumptions(ms, 'rating', "MS Total Score")

In [ ]:
check_assumptions(area_one, 'rating', "MS ratings Area 1")

In [ ]:
check_assumptions(area_two, 'rating', "MS ratings Area 2")

In [ ]:
check_assumptions(area_three, 'rating', "MS ratings Area 3")

In [ ]:
check_assumptions(area_four, 'rating', "MS ratings Area 4")

In [ ]:
check_assumptions(ssq, 'score', "SSQ Total Score")

## Correlations of the motion sickness variables

In [ ]:
## THIS IS ALSO IN THE QUESTIONNAIRE DATA/ PLOTS NB
fig, ax = plt.subplots(figsize=(4,4))
x = df[["H_AVG_MS", "FP_AVG_MS"]].mean(axis=1)
y = df["SSQ_TS_AVG"]
corr = stats.pearsonr(x, y)
sns.regplot(x=x, y=y, ax=ax, marker='o', color='navy')

fig.suptitle("Correlation Between SSQ and MS Ratings", fontsize=16, y=1.03)
ax.set_title("Pearsonr = {}, p-val = {}".format(corr[0].round(3), corr[1].round(3)), fontsize=13)
ax.set_ylabel("SSQ average",fontsize=16)
ax.set_xlabel("In-game MS average",fontsize=16)

sns.despine()
plt.show()

eff = None
if 0.1 <= corr[0] <= 0.3:
    eff = "small"
elif 0.3 < corr[0] <= 0.5:
    eff = "medium"
elif 0.5 < corr[0] <= 1.0:
    eff = "large"

if eff is not None:
    print("Effect size according to Cohen: " + eff)

# Embodiment

Subscales
- Spatial Presence (SP)
 - environmental location (EL)
 - possible actions (PA)
- Embodiment (EB)
 - self-location (SL)
 - agency (A)
 - ownership (O)
 
-> We're only interested in the embodiment part

## Effect of condition and mode on EB (total score)

In [ ]:
eb = pd.melt(df.reset_index(), id_vars=['ID', 'condition'], value_vars=['EB_EB_H', 'EB_EB_FP'])
eb = eb.rename(columns={"variable" : 'mode', "value" : "score"})
eb.replace({'mode': {'EB_EB_H': 'Hybrid', 'EB_EB_FP': 'First Person'},
               'condition': {1: 'Blob', 0: 'Avatar'}}, inplace=True)

eb_anova = pg.mixed_anova(dv='score', between='condition', within='mode', subject='ID', data=eb)
eb_anova

In [ ]:
summarize(eb, "score")

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
boxplot(eb, "score", "EB total score", ax)

In [ ]:
# if interaction is significant, create interaction plot
if eb_anova["p-unc"].tolist()[-1] < sig_level:
    fig, ax = plt.subplots(figsize=(4,4))
    interact_plot(eb, "score", "Interaction Condition x Mode (on EB total score)", ax=ax)

## Assumption Check Embodiment

In [ ]:
check_assumptions(eb, 'score', "EB Total Score")